# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation. To run this example Julia's `PyCall` package
needs to be able to find an installation of `pymatgen`.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

┌ Warning: `vendor()` is deprecated, use `BLAS.get_config()` and inspect the output instead
│   caller = npyinitialize() at numpy.jl:67
└ @ PyCall /home/runner/.julia/packages/PyCall/L0fLP/src/numpy.jl:67


Setup an LDA model and discretize using
a single k-point and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=(1, 1, 1))

PlaneWaveBasis discretization:
    Ecut                 : 5.0 Ha
    fft_size             : (32, 32, 32), 32768 total points
    kgrid type           : Monkhorst-Pack
    kgrid                : [1, 1, 1]
    num. irred. kpoints  : 1

    Discretized Model(lda_xc_teter93, 3D):
        lattice (in Bohr)    : [0         , 10.2631   , 10.2631   ]
                               [10.2631   , 0         , 10.2631   ]
                               [10.2631   , 10.2631   , 0         ]
        unit cell volume     : 2162.1 Bohr³
    
        atoms                : Si₁₆
        atom potentials      : ElementPsp(Si, psp="hgh/lda/si-q4")
    
        num. electrons       : 64
        spin polarization    : none
        temperature          : 0 Ha
    
        terms                : Kinetic()
                               AtomicLocal()
                               AtomicNonlocal()
                               Ewald()
                               PspCorrection()
                               

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.127006e+02     1.401050e+00
 * time: 0.4835391044616699
     1     1.052382e+01     8.782966e-01
 * time: 1.2294049263000488
     2    -1.219342e+01     9.209033e-01
 * time: 1.3055579662322998
     3    -3.419159e+01     6.887665e-01
 * time: 1.4442319869995117
     4    -4.783090e+01     5.044450e-01
 * time: 1.5300559997558594
     5    -5.703266e+01     1.986189e-01
 * time: 1.6152000427246094
     6    -5.983265e+01     1.239731e-01
 * time: 1.6911990642547607
     7    -6.090405e+01     4.694786e-02
 * time: 1.7698569297790527
     8    -6.133768e+01     6.186128e-02
 * time: 1.8525550365447998
     9    -6.162657e+01     3.956730e-02
 * time: 1.9171760082244873
    10    -6.183780e+01     2.716893e-02
 * time: 1.9805231094360352
    11    -6.198354e+01     1.664752e-02
 * time: 2.043645143508911
    12    -6.204665e+01     1.525308e-02
 * time: 2.1072211265563965
    13    -6.211260e+01     1.159852e-02
 * time: 2.18560600280

In [5]:
scfres.energies

Energy breakdown (in Ha):
    Kinetic             25.7671065
    AtomicLocal         -18.8557664
    AtomicNonlocal      14.8522637
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485367 
    Xc                  -19.3336818

    total               -62.261666460318